In [2]:
import pandas as pd
import altair as alt

alt.renderers.enable('html')

RendererRegistry.enable('html')

# Data setup

Just some data munging to get things ready for below

In [ ]:
data = pd.read_csv("data/daily.csv")

In [ ]:
pivot = pd.pivot_table(data, values='positive', index='state', columns='date')

In [ ]:
minnesota = data[data.state == 'MN']

# Minnesota curves

Sketches of Minnesota's Covid-19 growth curves based on positive tests, on both a linear and log scale.

In [ ]:
mn_curve_linear = alt.Chart(minnesota).mark_line().encode(
    x='dateChecked',
    y='positive'
)
mn_curve_linear

In [ ]:
mn_curve_log = alt.Chart(minnesota[minnesota.positive > 0]).mark_line().encode(
    alt.X('dateChecked'),
    alt.Y('positive', scale=alt.Scale(type='log', base=10))
)
mn_curve_log

# Minnesota doubling time

Defined as the natural log of 2 (~70) divided by the growth rate. NYT appears to define doubling time as the average daily rate of growth (as a percentage) over the last week.

In [ ]:
minnesota = minnesota.sort_values(['date'], ascending=[1])
minnesota['growth_rate'] = float(0.0)

# Calculate growth rates each day
for i in range(3, len(minnesota)):
    new = minnesota.iloc[i]['positive']
    old = minnesota.iloc[i-1]['positive']
    minnesota.iat[i, 10] = (new - old) / old
minnesota

In [ ]:
week_avg_growth_rate = minnesota.tail(7)['growth_rate'].mean() * 100 # As a percentage

# 70 is a constant representing the rough natural log of 2
doubling_time = 70 / week_avg_growth_rate

doubling_time

# Other states' curves

Same as above, but for other key states. In linear and log.

In [ ]:
filtered = data[(data.state == 'MN') |
                (data.state == 'WA') |
                (data.state == 'NY') |
                (data.state == 'IL') |
                (data.state == 'WI') |
                (data.state == 'CA')]

In [ ]:
all_curve_linear = alt.Chart(filtered).mark_line().encode(
    x='dateChecked',
    y='positive',
    color="state"
)
all_curve_linear

In [ ]:
all_curve_log = alt.Chart(filtered[filtered.positive > 0]).mark_line().encode(
    alt.X('dateChecked'),
    alt.Y('positive', scale=alt.Scale(type='log', base=10)),
    color="state"
)
all_curve_log

# Other states' doubling times

Same as above, but for other key states.

In [ ]:
for s in data['state'].unique():
    tmp = data[data.state == s].sort_values(['date'], ascending=[1])
    tmp['growth_rate'] = float(0.0)
    
    for i in range(3, len(tmp)):
        new = tmp.iloc[i]['positive']
        old = tmp.iloc[i-1]['positive']
        tmp.iat[i, 10] = (new - old) / old
    
    week_avg_growth_rate = tmp.tail(7)['growth_rate'].mean() * 100 # As a percentage
    doubling_time = 70 / week_avg_growth_rate

    print(s)
    print(doubling_time)